In [ ]:
import os
from PIL import Image
import numpy as np

# Define the input directory
dataset_path = "../mri_dataset/kaggle_3m/"
output_path = "../mri_dataset/super_resolved/"
X_train = []
y_train = []

# Define the target resolution (width, height)
target_resolution = (256, 256)

# Create the output directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Define a custom sorting function to sort filenames by numeric part
def sort_by_numeric(filename):
    return int(''.join(filter(str.isdigit, filename)))

# Function to downsample an image
def downsample_image(img, target_resolution):
    return img.resize(target_resolution, Image.LANCZOS)

# Iterate over TIFF files in the dataset
for patient in os.listdir(dataset_path):
    # Iterate over TIFF files for each patient
    for tif_file in sorted(os.listdir(os.path.join(dataset_path, patient)), key=sort_by_numeric):
        if "_mask" not in tif_file:  # Filter out "mask" images
            # Read original image
            original_image_path = os.path.join(dataset_path, patient, tif_file)
            original_img = Image.open(original_image_path)
            
            # Downsample the image
            downscaled_img = downsample_image(original_img, target_resolution)
            
            # Save the downsampled image to the output directory
            output_downscaled_image_path = os.path.join(output_path, patient, tif_file)
            os.makedirs(os.path.dirname(output_downscaled_image_path), exist_ok=True)
            downscaled_img.save(output_downscaled_image_path)
            
            # Convert image to numpy array and append to training data
            original_img_array = np.array(original_img)
            downscaled_img_array = np.array(downscaled_img)
            X_train.append(downscaled_img_array)
            y_train.append(original_img_array)

# Convert training data to numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

# Normalize the pixel values to the range [0, 1]
X_train = X_train.astype('float32') / 255.0
y_train = y_train.astype('float32') / 255.0


: 

In [2]:
super_resolution_model = build_super_resolution_model(input_shape)


In [10]:
len(patient_images["TCGA_CS_4941_19960909"][0])

2

In [11]:
X_train = []
y_train = []

# Define the target resolution (width, height)
target_resolution = (256, 256)

# Create the output directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Define a custom sorting function to sort filenames by numeric part
def sort_by_numeric(filename):
    return int(''.join(filter(str.isdigit, filename)))

# Function to downsample an image
def downsample_image(img, target_resolution):
    return img.resize(target_resolution, Image.LANCZOS)

# Iterate over patients in the dataset
for patient in os.listdir(dataset_path):
    patient_images[patient] = []
    
    # Iterate over TIFF files for each patient
    for tif_file in sorted(os.listdir(os.path.join(dataset_path, patient)), key=sort_by_numeric):
        if "_mask" not in tif_file:  # Filter out "mask" images
            # Read original image
            original_image_path = os.path.join(dataset_path, patient, tif_file)
            original_img = Image.open(original_image_path)
            
            # Downsample the image
            downscaled_img = downsample_image(original_img, target_resolution)
            
            # Save the downsampled image to the output directory
            output_downscaled_image_path = os.path.join(output_path, patient, tif_file)
            os.makedirs(os.path.dirname(output_downscaled_image_path), exist_ok=True)
            downscaled_img.save(output_downscaled_image_path)
            
            # Convert image to numpy array and append to patient images list
            original_img_array = np.array(original_img)
            downscaled_img_array = np.array(downscaled_img)
            patient_images[patient].append([original_img_array, downscaled_img_array])
            
            # Append to training data
            X_train.append(downscaled_img_array)
            y_train.append(original_img_array)

# Convert training data to numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

# Normalize the pixel values to the range [0, 1]
X_train = X_train.astype('float32') / 255.0
y_train = y_train.astype('float32') / 255.0

: 

In [ ]:
super_resolution_model.compile(optimizer='adam', loss='mse')


In [ ]:
super_resolution_model.fit(X_train, y_train, epochs=10, batch_size=32)

# Perform super-resolution on new downsampled images and save the super-resolved images
for patient, image_list in patient_images.items():
    for i, (_, downscaled_img) in enumerate(image_list):
        super_resolved_img = super_resolution_model.predict(downscaled_img.reshape(1, *target_resolution, 3))
        super_resolved_img = np.squeeze(super_resolved_img)  # Remove batch dimension
        output_super_resolved_image_path = os.path.join(output_path, patient, f"super_resolved_{i}.png")
        os.makedirs(os.path.dirname(output_super_resolved_image_path), exist_ok=True)
        Image.fromarray((super_resolved_img * 255).astype('uint8')).save(output_super_resolved_image_path)